In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

In [2]:
#"https://www.dandalinvoa.com/wasanni?p={0}", 101
#"https://www.dandalinvoa.com/nishadi?p={0}", 60
#"https://www.dandalinvoa.com/rayuwa?p={0}", 77
sources =  [
            "https://www.dandalinvoa.com/labarai?p={0}"
           ]

In [3]:
sources_page_limit = {0:101}

In [4]:
def init_driver():
    """
    This func initializes the webdriver and disables images
    A wait is initialized with a 5 second timeout
    """
    chrome_path = 'C:\\Users\\NiniolaAdegboyega\\Downloads\\chromedriver_win32\\chromedriver.exe'
    options = Options()
    # disables images in browser. For faster loading. 
    prefs = {"profile.managed_default_content_settings.images" : 2}
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(chrome_path, options=options)
    # define a generic wait to be used throughout
    driver.wait = WebDriverWait(driver, 5)

    return driver
        
def to_txt(sentences, file):
    """
    This func splits the sentences by fullstops then
    writes each individual sentences on a new line.
    """
    sentence_split = filter(None, sentences.split("."))
    
    for s in sentence_split:
        #print(s.strip())
        file.write(s.strip() + "\n")

def on_article_action(driver, file):
    """
    This func controls the actions that occurs when an
    article is clicked and returns to the article list page.
    """
    # an empty string is initialized to hold content of each article
    text = ""

    time.sleep(2)

    content = driver.page_source
    soup = bs(content)

    # loops over elements in soup that contain words and extend text
    for pre in soup.findAll("p"):

        text += pre.text

    # writes title of article
    file.write(soup.find("title").text)
    # writes body of article
    to_txt(text, file)

    # returns to article list page
    driver.back()
        
def download_articles(driver, sources):
    """
    * This func traverses the entire website first by column
    then by page then by article. 
    * Each individual column is written into a new text file. 
    """
    scroll_link = "/html/body/div[2]/div/div[4]/div/div/div/div/div/div[2]/p/a"
    
    # initialize empty list to hold those articles that weren't collected
    errored_out = list()
    
    # loops over the individual sources
    for i, source in enumerate(sources):
        
        # initialize empty list to hold those j that fails due TimeOutException
        err = list()

        filename = "../data/dandalinvoa/Source_{0}.txt" 

        file = open(filename.format(i), "w+", encoding="utf-8")
        
        print(f"Starting Source {i}")

        # first article on page has a different xpath
        article_0 = "/html/body/div[2]/div/div[4]/div/div/div/div/div/div[2]/div/ul[1]/li/div/div/div/a/h4"
        # xpath string template for each article
        article_n = "/html/body/div[2]/div/div[4]/div/div/div/div/div/div[2]/div/ul[2]/li[{0}]/div/div"

        # loops over the pages of the sources
        for j in range(0, sources_page_limit[i]):

            driver.get(source.format(j))
            # wait until the first article element on the page is loaded. 
            driver.wait.until(EC.visibility_of_element_located((By.XPATH, article_n.format(1))))

            # clicks the zeroth article on the page
            #driver.find_element_by_xpath(article_0).click()

            #on_article_action(driver, file)

            # wait until the first article element on the page is loaded. 
            #driver.wait.until(EC.visibility_of_element_located((By.XPATH, article_n.format(1))))

            # loops over the articles on each page
            for k in range(1, 11):
               
                if k > 2:
                    # if article number is greater than 2, scroll the second
                    # article before it into view. Avoids NoSuchElementException
                    driver.execute_script("arguments[0].scrollIntoView();",
                    driver.find_element_by_xpath(article_n.format(k-2)))
                
                try:
                    # clicks article
                    driver.find_element_by_xpath(article_n.format(k)).click()
                # if network slows down or shuts off.
                except TimeoutException:
                    err.append(k)
                    time.sleep(10)
                # last page may not have up to 11 articles
                except NoSuchElementException:
                    continue
                 
                
                on_article_action(driver, file)
                 
                # waits for element of next article to be loaded 
                if k < 11:
                    driver.wait.until(EC.visibility_of_element_located((By.XPATH, article_n.format(k+1))))
        
        print("The following articles failed: ", errored_out.append((i, err)))
        print(f"O ti pari Source {i}")
        # closes file used for storage.
        file.close()
        
    print("Opin!")
        
if __name__ == "__main__":
    
    driver = init_driver()
    download_articles(driver, sources)
    driver.close()

Starting Source 0
The following articles failed:  None
O ti pari Source 0
Opin!


In [6]:
def remove_blank_lines(filename):
    """
    This func removes blank lines from txt file
    """
    file = open(filename[:-4] + "_1.txt", "w+", encoding="utf-8")

    with open(filename, encoding="utf-8") as f:
        file.write("".join(line for line in f if not line.isspace()))
    
    file.close()

In [7]:
remove_blank_lines("Labarai.txt")